<a href="https://colab.research.google.com/github/Samarthsri1608/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install pdfplumber
!pip install google-genai

In [2]:
import requests
import pdfplumber
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
from google import genai

In [3]:
# PDF link of Harry Potter and Prisoner of Azkaban
pdf_url = 'https://vidyaprabodhinicollege.edu.in/VPCCECM/ebooks/ENGLISH%20LITERATURE/Harry%20potter/(Book%203)%20Harry%20Potter%20And%20The%20Prisoner%20Of%20Azkaban_001.pdf'


In [4]:
response = requests.get(pdf_url)
with open('document.pdf', 'wb') as f:
    f.write(response.content)

In [5]:
full_text = ""
with pdfplumber.open("document.pdf") as pdf:
  for page in pdf.pages:
    full_text += page.extract_text()

In [ ]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [7]:
chunks = full_text.split("\n")
embeddings = embedder.encode(chunks)

In [8]:
def retrieve(query, top_n=3):
    query_vector = embedder.encode([query])
    sims = cosine_similarity(query_vector, embeddings)[0]
    top_idx = np.argsort(sims)[-top_n:][::-1]
    return [chunks[i] for i in top_idx]


In [9]:
GEMINI_API_KEY = "<>"
client = genai.Client(api_key=GEMINI_API_KEY)

In [10]:
def respond(query):
  Context = "\n".join(retrieve(query, top_n=3))
  print(f"Recieved Context = {Context}")
  response = client.models.generate_content(
      model="gemini-2.0-flash-lite", contents=f"""You are an assistant, answer the query based on the given context.
      Context: {Context}, query = {query}
      """
  )
  return response.text

In [ ]:
print(respond("<>"))